In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
import ast

warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)

## Data

In [3]:
ret = pd.read_excel("midterm_2.xlsx", sheet_name=0, index_col=0)
factors = pd.read_excel("midterm_2.xlsx", sheet_name=1, index_col=0)
ret.head()

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.0084,-0.0016,0.1550,0.1146,0.1379,-0.0280,0.2956,0.2296,0.0223,0.0988,-0.0185
2009-05-31,0.0541,0.1631,0.1599,0.1324,0.0290,-0.0203,0.0232,0.0544,0.0283,0.0589,0.0204
2009-06-30,0.0045,-0.0269,-0.0231,-0.0146,0.0329,-0.0062,-0.0255,0.0448,-0.0040,-0.0013,0.0014
2009-07-31,0.0313,0.0186,0.1102,0.1004,0.0692,0.0083,0.1058,0.1433,0.0154,0.0746,0.0009
2009-08-31,0.0072,-0.0408,-0.0136,0.0446,-0.0174,0.0072,0.1315,0.0330,-0.0046,0.0365,0.0080


In [4]:
factors.head()

,MKT,RMW
Date,,
2009-04-30,0.1018,0.0131
2009-05-31,0.0521,-0.0078
2009-06-30,0.0043,-0.0141
2009-07-31,0.0772,-0.0046
2009-08-31,0.0333,-0.0302


## Functions

In [74]:
def get_capm_matrics(targets, regressors, add_constant=True, annualize_factor=12):

    result = pd.DataFrame(index=targets.columns)
    resid_matrix = pd.DataFrame(columns=targets.columns)
    t_p_value = pd.DataFrame(index=targets.columns)

    if add_constant:
        X = sm.add_constant(regressors)
    else:
        X = regressors.copy()
    for column in targets.columns:
        y = targets[[column]]
        model = sm.OLS(y, X, missing='drop').fit()
        if add_constant:
            result.loc[column, "alpha"] = model.params['const'] * \
                annualize_factor
        result.loc[column, regressors.columns] = model.params[regressors.columns]

        result.loc[column, "R-squared"] = model.rsquared

        resid_matrix[column] = model.resid
        if add_constant:
            t_p_value.loc[column, "t-value"] = model.tvalues['const']
            t_p_value.loc[column, "p-value"] = model.pvalues['const']

    return result, resid_matrix, t_p_value


def cal_excess_tangency(df_ex, annual_factor):
    N = df_ex.shape[1]

    Sigma = df_ex.cov() * annual_factor
    Sigma_inv = np.linalg.inv(Sigma)
    mean_ex = df_ex.mean() * annual_factor

    w_t = 1/(np.ones(N) @ Sigma_inv @ mean_ex) * Sigma_inv @ mean_ex
    return pd.Series(w_t, index=df_ex.columns)


## Q1 Short Answer

### 1

-	For the market factor, we can calculate it by the value-weighted method. In this way, we calculated value-neutral market factor
-	For size factor, we can also calculate value-weighed size. And the other way is first to divide stocks into different size groups and build value factor that are neutral to size. 

### 2

-	In the 1990s, DFA saw value stocks greatly under-perform growth stocks, partially driven by the incredible returns of tech stocks. In other periods, the value factor has a relatively good performance. 
-	Other than that, value factor has a small correlation to other factors, and adds benefits to the distribution of the portfolio. 


### 3

-	Once the CMA factor, which was largely correlated with HML, is taken out of the tangency portfolio, and doing this four-factor model by AQR, Value factor emerges to be the most significant as it has really low correlations with the remaining factors. HML and RMW had the 2nd highest correlation previously, and the removal of RMW also contributes to a larger weight for HML in the tangency portfolio.
-	SMB now has the highest correlation between the 4 factors with MKT and thus is penalized for it and has the lowest weight in the tangency portfolio
-	All these style factors can come in handy for both a linear factor decomposition and in a linear pricing model, as long as the combination of these factors do not posit heavy correlations. All the factors have posted positive risk premia for a longer duration, but due regime changes and changes in market microstructure resulting from advancing technologies and changing investor behavior, some factors might now prove to be redundant or might not display positive risk premia for a longer duration.

### 4

- Yes and No. If CAPM worked perfectly, the market portfolio is the tangency portfolio, which has the highest Sharpe ratio. And all assets are on the same SML so that all asset has the same Treynor ratio that is equal to the market factor’s Treynor ratio. 


### 5

-	Long-only momentum has a high correlation to market factor, while long-short momentum has a negative correlation to the market factor. (biggest difference)
-	Besides that, long-only momentum has a higher mean return and the quite same volatility as the long-short momentum factor and thus has a higher Sharpe ratio. 


### 6

-	The r-squared of BTC auto-regression is 0.1, which shows that BTC has relatively strong momentum. 
-	However, just for one asset BTC, it is impossible to buy the winner and sell the loser. If we are investing in the cryptocurrency market, momentum may be a possible strategy. - 


### 7

-	MKT factor can the entire market have a positive net exposure.(if you take MKT=1)
-	UMD factor is constructed to have a zero net value portfolio. 


### 8

-	To have a good replication in the time-series perspective, the residuals are uncorrelated across regressors. That is, the factors completely describe return co-movement. 
-	To have no arbitrage. 


### 9

- Yes. If we have a perfect LFP, the factors describe expected returns across assets. However, for the time series model, the set of factors may not explain realized returns across time. 


## Q2 Pricing Model 
### 1 Estimate the time-series test of the pricing model.

#### (a) (8pts) Report...

(for each asset) annualized alpha, beta, and r-squared.

In [6]:
result, resid, _ = get_capm_matrics(
    ret, factors, add_constant=True, annualize_factor=12)
result

,alpha,MKT,RMW,R-squared
BWX,-0.0406,0.2363,0.1024,0.2047
DBC,-0.0520,0.5932,-0.1086,0.2568
EEM,-0.0689,0.9436,-0.0948,0.5601
EFA,-0.0582,0.9313,-0.0775,0.7622
HYG,-0.0004,0.4532,0.0362,0.5886
IEF,0.0240,-0.0818,0.1693,0.0820
IYR,0.0010,0.9134,0.2731,0.5514
PSP,-0.0659,1.3408,-0.1098,0.8393
QAI,-0.0237,0.2782,0.0028,0.7174
SPY,0.0006,0.9629,0.1259,0.9924


In [9]:
print("Annualized MAE: " + str(round(result["alpha"].abs().mean(), 4)))

Annualized MAE: 0.0311


In [10]:
print("Mean of the r-squared statistics: " +
      str(round(result["R-squared"].mean(), 4)))

Mean of the r-squared statistics: 0.5155


#### (b) (7pts) If the pricing model worked perfectly, what would these statistics be?

- The MAE should equal to zero. 
- We do not care much about R-squared in time-series regression.

#### (c) (5pts) What do you conclude from the time-series test of this model? That is, give some interpretation as to how well it prices the assets, and be specific.

- It is not so well as pricing cause the MAE do not equal to zero.
- We can have more test on alpha for the Joint Distribution.

### 2 Estimate the cross-sectional test of the pricing model. Include an intercept in the cross-sectional regression.

(a) (8pts) Report the...

In [42]:
y = ret.mean().to_frame("result")
result_cs, resid_cs, _ = get_capm_matrics(
    y, result[["MKT", "RMW"]], add_constant=True, annualize_factor=12)

In [43]:
print("Annualized intercept: " + str(round(result_cs.alpha[0], 4)))

Annualized intercept: -0.0214


In [44]:
print("annualized factor premia: ")
result_cs[["MKT", "RMW"]]*12

annualized factor premia: 


,MKT,RMW
result,0.1186,0.2101


In [45]:
print("Annualized intercept: " + str(round(result_cs["R-squared"][0], 4)))

Annualized intercept: 0.9055


In [46]:
print("Annualized mean-absolute error: " +
      str(round(resid_cs.abs().mean()[0] * 12, 4)))

Annualized mean-absolute error: 0.0109


### (b) (7pts) If the pricing model worked perfectly, what would these statistics be?

- Intercept should equal to 0.
- Factor premia should equal to time-series factor premia.
- r-squared should equal to 1.
- Annualized mean-absolute error should less than(or equal to, is related to whether include an intercept or not) the MAE of time-series alpha.

### (c) (5pts) What do you conclude from the cross-sectional test of this model? That is, give some interpretation as to how well it prices the assets, and be specific.

- The R-square is pretty high (at around 0.91)
- However the intercept is not zero and means it is not so well when used to predict the reisk premia

### 3 Is the RMW factor more lucrative according to the cross-sectional or time-series estimates?

- Way more attractive according to the cross-sectional estimation. 

In [50]:
print("time-series factor premia: ")
(factors.mean() * 12).to_frame("time-series")

time-series factor premia: 


,time-series
MKT,0.1385
RMW,0.0270


In [51]:
print("cross-sectional factor premia: ")
result_cs[["MKT", "RMW"]] * 12

cross-sectional factor premia: 


,MKT,RMW
result,0.1186,0.2101


### 4 Make a chart showing the annualized 11 asset premia, (that is, expected excess returns,) as implied by the

- sample average

In [53]:
(ret.mean() * 12).to_frame("sample average")


,sample average
BWX,-0.0051
DBC,0.0272
EEM,0.0592
EFA,0.0687
HYG,0.0633
IEF,0.0172
IYR,0.1349
PSP,0.1168
QAI,0.0149
SPY,0.1373


In [90]:
pd.DataFrame((factors.mean().values * result[["MKT", "RMW"]
                                             ].values * 12).sum(axis=1), index=ret.columns, columns=["ts estimated"])


,ts estimated
BWX,0.0355
DBC,0.0792
EEM,0.1281
EFA,0.1269
HYG,0.0637
IEF,-0.0068
IYR,0.1339
PSP,0.1827
QAI,0.0386
SPY,0.1367


In [73]:
cs_estimation = pd.DataFrame((result_cs[["MKT", "RMW"]].values * result[["MKT", "RMW"]].values * 12).sum(axis=1), \
    index=ret.columns, columns=["cs estimated"])
cs_estimation

,cs estimated
BWX,0.0495
DBC,0.0476
EEM,0.0920
EFA,0.0942
HYG,0.0614
IEF,0.0259
IYR,0.1657
PSP,0.1360
QAI,0.0336
SPY,0.1407


## 3 Applications to Previously Seen Topics

### 1 Calculate the tangency portfolio of the 11 excess return assets based on sample data. Report the

In [77]:
tan_weight = cal_excess_tangency(ret, annual_factor=12).to_frame("weight")
tan_weight

,weight
BWX,-16.6834
DBC,2.6617
EEM,5.0023
EFA,-1.6528
HYG,15.8455
IEF,31.6714
IYR,-4.2510
PSP,-6.9802
QAI,-60.2653
SPY,33.1711


In [86]:
ret_tan = (tan_weight.values.T * ret.values).sum(axis=1)
sharpe_ratio = ret_tan.mean() * np.sqrt(12) / ret_tan.std()
print("Sharpe ratio: " + str(round(sharpe_ratio, 4)))

Sharpe ratio: 1.847


### 2 Re-do the previous question regarding the tangency portfolio. But this time, utilize the expected returns as implied by the cross-sectional estimation of the pricing model in the tangency portfolio estimation. That is, use the cross-sectional premia shown in your table from Question 2.4.

In [99]:
sigma = ret.cov()
sigma

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,0.0005,4.2576e-04,0.0008,0.0007,3.3350e-04,1.5444e-04,5.5550e-04,0.0008,2.2133e-04,0.0005,2.0870e-04
DBC,0.0004,2.7385e-03,0.0017,0.0015,6.3672e-04,-3.3199e-04,8.6695e-04,0.0017,4.1598e-04,0.0011,9.4439e-05
EEM,0.0008,1.6852e-03,0.0031,0.0023,1.0679e-03,-1.8893e-04,1.8671e-03,0.0029,6.5219e-04,0.0018,2.3155e-04
EFA,0.0007,1.4611e-03,0.0023,0.0022,9.5238e-04,-1.7786e-04,1.7835e-03,0.0028,5.8239e-04,0.0018,1.8060e-04
HYG,0.0003,6.3672e-04,0.0011,0.0010,6.8098e-04,-2.5072e-05,1.0679e-03,0.0014,2.9115e-04,0.0009,1.3303e-04
IEF,0.0002,-3.3199e-04,-0.0002,-0.0002,-2.5072e-05,3.0909e-04,2.9916e-05,-0.0002,1.8207e-06,-0.0002,1.8205e-04
IYR,0.0006,8.6695e-04,0.0019,0.0018,1.0679e-03,2.9916e-05,2.9500e-03,0.0027,5.0914e-04,0.0017,3.0923e-04
PSP,0.0008,1.6971e-03,0.0029,0.0028,1.4069e-03,-2.1450e-04,2.6730e-03,0.0042,7.9101e-04,0.0025,2.8216e-04
QAI,0.0002,4.1598e-04,0.0007,0.0006,2.9115e-04,1.8207e-06,5.0914e-04,0.0008,2.1143e-04,0.0005,9.7780e-05
SPY,0.0005,1.1278e-03,0.0018,0.0018,8.5164e-04,-1.5695e-04,1.7425e-03,0.0025,5.2289e-04,0.0018,1.7072e-04


In [100]:
factors.mean()

MKT    0.0115
RMW    0.0022
dtype: float64

In [106]:
sigma = ret.cov()
w_tan_unscaled = np.linalg.inv(sigma) @ cs_estimation.values
wtan = pd.DataFrame(w_tan_unscaled / w_tan_unscaled.sum(), 
                    index = ret.columns, 
                    columns = ['Tangency Weights'])
wtan

,Tangency Weights
BWX,0.3370
DBC,-0.0287
EEM,-0.0203
EFA,-0.1755
HYG,0.2264
IEF,-0.1946
IYR,0.0735
PSP,-0.5109
QAI,-1.3843
SPY,1.3865


### 3 Assume lognormal, iid returns. Suppose for this question that, (in log returns,) the expected excess return of RMW is 3% with a 7% volatility.

In [111]:
import scipy.stats as stats
mean = 0.03 * 5
var = 0.07 * np.sqrt(5)
1 - stats.norm.cdf(-mean/var)

0.8310479904484973